PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, ChatGPT)

In [1]:
%pip install -q openai
%pip install -q langchain
%pip install -q -U langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

In [3]:
%pip install -q pypdf
%pip install -q chromadb
%pip install -q tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
# PDF 로드
loader = PyPDFLoader("data/how-to-use-AI-in-6industries.pdf")
documents = loader.load()

In [7]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [8]:
# OpenAI Embeddings 적용
embeddings = OpenAIEmbeddings()

# Chroma 벡터 저장소 생성
vector_store = Chroma.from_documents(texts, embeddings)
# 벡터 저장소에서 검색하기
# search_type="similarity" 옵션은 retrieval 객체에서 유사성 검색을 사용하여 질문 vector와 가장 유사한 문장 vector를 선택함
# search_kwargs 옵션은 vector 저장소에서 Prompt 2개의 텍스트 덩어리로 보내려는 것을 의미함
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [49]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# ChatOpenAI는 기본모델인  gpt-3.5-turbo 사용하고
# temperature=0은 보수적인 지문에 대한 답변을 내고, temperature=1 다양한 답변을 낼 수 있음 
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# RetrievalQA가 실질적인 RAG를 수행하는 객체
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # 기존 retriever 유지
    return_source_documents=True
)

In [16]:
query = "언더라이팅"
query_result = chain.invoke(query)
print(type(query_result))
print(query_result)

<class 'dict'>
{'query': '언더라이팅', 'result': '언더라이팅(underwriting)은 보험사가 보험계약 인수 여부를 판단하는 최종 단계를 말합니다.', 'source_documents': [Document(metadata={'creationdate': '2023-08-10T15:32:26+09:00', 'creator': 'Adobe InDesign 18.3 (Windows)', 'keywords': '한국 딜로이트 그룹, 딜로이트, 딜로이트 인사이트, 딜로이트 보고서, 딜로이트 미래 전망, 미래 산업 전망, 인공지능 전략, AI', 'moddate': '2023-08-10T16:13:14+09:00', 'page': 36, 'page_label': '37', 'producer': 'Adobe PDF Library 17.0', 'source': 'how-to-use-AI-in-6industries.pdf', 'title': '인공지능(AI) 활용서 리포트', 'total_pages': 74, 'trapped': '/False'}, page_content='인공지능(AI) 활용서: 6대 산업별 AI 활용사례37\n(3) 보험 언더라이팅 업무 고도화\nAI와 머신러닝을  사용하여  언더라이팅  프로세스  및 위험 평가를  개선하고 , 의사 결정 시간을  단축하며 , 고객 경험 및 가입률을  개선할  \n수 있다.\n이슈 및 개선 기회\n많은 보험사들이  지난 몇 년 동안 고객 온보딩과  보험계약을  디지털화하기  위해 상당한  투자를  해왔음에도  불구하고 , 언더라이팅  현대화를  \n실질적으로  확장하지  못해 디지털화는  느리고  점진적으로  진행되고  있다.\nAI가 지원하는 방식\n █언더라이팅  프로세스의  자동화\n텍스트  마이닝 (text mining) 과 자연어  처리를  사용하여  사람의  개입이  필요하지  않은 자동화된  언더라이팅  플랫폼을  구현할  수 있으며 , \n이를 통해 보험 신청서  처리에  소요되

In [12]:
query_result['result']

'언더라이팅(underwriting)은 보험사가 보험계약 인수 여부를 판단하는 최종 단계를 말합니다.'

In [13]:
for key in query_result.keys():
    print(key)

query
result
source_documents


In [14]:
print(type(query_result['source_documents']))
type(query_result['source_documents'][0])

<class 'list'>


langchain_core.documents.base.Document

In [18]:
doc = query_result['source_documents'][0]
print(type(doc))
doc_dict = doc.model_dump()
print(type(doc_dict))
print(doc_dict)

<class 'langchain_core.documents.base.Document'>
<class 'dict'>
{'id': None, 'metadata': {'creationdate': '2023-08-10T15:32:26+09:00', 'creator': 'Adobe InDesign 18.3 (Windows)', 'keywords': '한국 딜로이트 그룹, 딜로이트, 딜로이트 인사이트, 딜로이트 보고서, 딜로이트 미래 전망, 미래 산업 전망, 인공지능 전략, AI', 'moddate': '2023-08-10T16:13:14+09:00', 'page': 36, 'page_label': '37', 'producer': 'Adobe PDF Library 17.0', 'source': 'how-to-use-AI-in-6industries.pdf', 'title': '인공지능(AI) 활용서 리포트', 'total_pages': 74, 'trapped': '/False'}, 'page_content': '인공지능(AI) 활용서: 6대 산업별 AI 활용사례37\n(3) 보험 언더라이팅 업무 고도화\nAI와 머신러닝을  사용하여  언더라이팅  프로세스  및 위험 평가를  개선하고 , 의사 결정 시간을  단축하며 , 고객 경험 및 가입률을  개선할  \n수 있다.\n이슈 및 개선 기회\n많은 보험사들이  지난 몇 년 동안 고객 온보딩과  보험계약을  디지털화하기  위해 상당한  투자를  해왔음에도  불구하고 , 언더라이팅  현대화를  \n실질적으로  확장하지  못해 디지털화는  느리고  점진적으로  진행되고  있다.\nAI가 지원하는 방식\n █언더라이팅  프로세스의  자동화\n텍스트  마이닝 (text mining) 과 자연어  처리를  사용하여  사람의  개입이  필요하지  않은 자동화된  언더라이팅  플랫폼을  구현할  수 있으며 , \n이를 통해 보험 신청서  처리에  소요되는  시간을  크게 단축시킬  수 있다. \n █더 간편하고  사용자  친화적인  보험가입

In [43]:
for k in doc_dict:
    print(k)

id
metadata
page_content
type


In [19]:
doc_dict['metadata']['title']

'인공지능(AI) 활용서 리포트'

In [50]:
query = "소비자 부문 AI 활용사례 무엇인가요?"
query_result = chain.invoke(query)
print(query_result)

{'query': '소비자 부문 AI 활용사례 무엇인가요?', 'result': '소비자 부문에서의 AI 활용사례로는 비디오 콘텐츠 분석을 통한 데이터 수익화, 오디오/비디오 마이닝, 그리고 시청자 감정 감지를 통한 광고 분석 등이 있습니다. 이를 통해 기업들은 소비자들의 행동과 선호도를 더 잘 이해하고, 이를 바탕으로 맞춤형 서비스를 제공하거나 잠재적인 문제를 예방하는 데 활용할 수 있습니다.', 'source_documents': [Document(metadata={'creationdate': '2023-08-10T15:32:26+09:00', 'creator': 'Adobe InDesign 18.3 (Windows)', 'keywords': '한국 딜로이트 그룹, 딜로이트, 딜로이트 인사이트, 딜로이트 보고서, 딜로이트 미래 전망, 미래 산업 전망, 인공지능 전략, AI', 'moddate': '2023-08-10T16:13:14+09:00', 'page': 13, 'page_label': '14', 'producer': 'Adobe PDF Library 17.0', 'source': 'how-to-use-AI-in-6industries.pdf', 'title': '인공지능(AI) 활용서 리포트', 'total_pages': 74, 'trapped': '/False'}, page_content='인공지능(AI) 활용서: 6대 산업별 AI 활용사례14\n02\n소비자 부문 AI 활용서\n소비자 (Consumer) 산업은  소비재부터  소매, 자동차 , 숙박, 외식, 여행, 이동과  수송(운송)까지 광범위한  부문을  포함한다 . 서로 별개\n의 사업처럼  보이는  이들 부문은  고객인  소비자에게  상품과  서비스를  제공한다는  강력하고도  변치 않는 공통점이  있다. 그렇기에  현\n재와 미래에  풀어야  할 비즈니스  과제 또한 공유한다 .\n소비자  관련 기업들은  AI의 도움으로  운영 효율성을  끌어올리고  다양한  AI 활용사례를  십분 활용해

In [51]:
print(query_result['result'])

소비자 부문에서의 AI 활용사례로는 비디오 콘텐츠 분석을 통한 데이터 수익화, 오디오/비디오 마이닝, 그리고 시청자 감정 감지를 통한 광고 분석 등이 있습니다. 이를 통해 기업들은 소비자들의 행동과 선호도를 더 잘 이해하고, 이를 바탕으로 맞춤형 서비스를 제공하거나 잠재적인 문제를 예방하는 데 활용할 수 있습니다.


In [43]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

system_template = """Use the following pieces of context to answer the user's question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), 
use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{question}")
])


In [44]:

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "summaries",
}

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # GPT-4를 사용하려면 model="gpt-4"

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    input_key="question"
)


In [45]:
query = "소비자 부문 AI 활용사례 무엇인가요?"
query_result = chain.invoke({"question": query})
print(type(query_result))
print(query_result)

<class 'dict'>
{'question': '소비자 부문 AI 활용사례 무엇인가요?', 'result': '**소비자 부문 AI 활용사례**는 소비자와 관련된 기업들이 AI 기술을 활용하여 운영 효율성을 높이고 가치를 창출하는데 활용하는 사례들을 다룹니다. 이를 통해 기업들은 데이터 분석, 고객 서비스 개선, 마케팅 전략 수립, 비즈니스 프로세스 최적화 등 다양한 영역에서 AI를 적극적으로 활용하고 있습니다. 소비자 부문 AI 활용사례에 대한 자세한 내용은 **인공지능(AI) 활용서: 6대 산업별 AI 활용사례** 문서를 참고하시면 됩니다. \n\n**SOURCES:** 인공지능(AI) 활용서: 6대 산업별 AI 활용사례14', 'source_documents': [Document(metadata={'creationdate': '2023-08-10T15:32:26+09:00', 'creator': 'Adobe InDesign 18.3 (Windows)', 'keywords': '한국 딜로이트 그룹, 딜로이트, 딜로이트 인사이트, 딜로이트 보고서, 딜로이트 미래 전망, 미래 산업 전망, 인공지능 전략, AI', 'moddate': '2023-08-10T16:13:14+09:00', 'page': 13, 'page_label': '14', 'producer': 'Adobe PDF Library 17.0', 'source': 'how-to-use-AI-in-6industries.pdf', 'title': '인공지능(AI) 활용서 리포트', 'total_pages': 74, 'trapped': '/False'}, page_content='인공지능(AI) 활용서: 6대 산업별 AI 활용사례14\n02\n소비자 부문 AI 활용서\n소비자 (Consumer) 산업은  소비재부터  소매, 자동차 , 숙박, 외식, 여행, 이동과  수송(운송)까지 광범위한  부문을  포함한다 . 서로 별개\n의 사업처럼  보이는  이들 부문은  고객인  소비자에게  상품과  서비스를  

In [46]:
print(query_result['result'])

**소비자 부문 AI 활용사례**는 소비자와 관련된 기업들이 AI 기술을 활용하여 운영 효율성을 높이고 가치를 창출하는데 활용하는 사례들을 다룹니다. 이를 통해 기업들은 데이터 분석, 고객 서비스 개선, 마케팅 전략 수립, 비즈니스 프로세스 최적화 등 다양한 영역에서 AI를 적극적으로 활용하고 있습니다. 소비자 부문 AI 활용사례에 대한 자세한 내용은 **인공지능(AI) 활용서: 6대 산업별 AI 활용사례** 문서를 참고하시면 됩니다. 

**SOURCES:** 인공지능(AI) 활용서: 6대 산업별 AI 활용사례14


In [32]:
query_result['source_documents']

[Document(metadata={'creationdate': '2023-08-10T15:32:26+09:00', 'creator': 'Adobe InDesign 18.3 (Windows)', 'keywords': '한국 딜로이트 그룹, 딜로이트, 딜로이트 인사이트, 딜로이트 보고서, 딜로이트 미래 전망, 미래 산업 전망, 인공지능 전략, AI', 'moddate': '2023-08-10T16:13:14+09:00', 'page': 13, 'page_label': '14', 'producer': 'Adobe PDF Library 17.0', 'source': 'how-to-use-AI-in-6industries.pdf', 'title': '인공지능(AI) 활용서 리포트', 'total_pages': 74, 'trapped': '/False'}, page_content='인공지능(AI) 활용서: 6대 산업별 AI 활용사례14\n02\n소비자 부문 AI 활용서\n소비자 (Consumer) 산업은  소비재부터  소매, 자동차 , 숙박, 외식, 여행, 이동과  수송(운송)까지 광범위한  부문을  포함한다 . 서로 별개\n의 사업처럼  보이는  이들 부문은  고객인  소비자에게  상품과  서비스를  제공한다는  강력하고도  변치 않는 공통점이  있다. 그렇기에  현\n재와 미래에  풀어야  할 비즈니스  과제 또한 공유한다 .\n소비자  관련 기업들은  AI의 도움으로  운영 효율성을  끌어올리고  다양한  AI 활용사례를  십분 활용해  가치를  창출하고자  적극적으로  \n방법을  모색하고  있다. 하지만  AI 도입과  성숙도  수준은  기업마다  매우 다르다 . 그 이유 또한 데이터  품질 및 복잡성에  따른 확장성  문\n제, 조직 구성 및 인력 부족 문제, 신뢰 부족 문제 등 다양하다 .\n그 중에서도  가장 공통적이면서도  해결하기  힘든 난제는  구상을  현실화해  규모를  키우는  것이다 . 소비자  관련 기업들  상당수는  대\n규모 레거시 (leg

In [33]:
for doc in query_result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

내용 : 인공지능(AI) 활용서: 6대 산업별 AI 활용사례14 02 소비자 부문 AI 활용서 소비자 (Consumer) 산업은  소비재부터  소매, 자동차 , 숙박, 외식, 여행, 이동과
파일 : how-to-use-AI-in-6industries.pdf
페이지 : 13
내용 : 인공지능(AI) 활용서: 6대 산업별 AI 활용사례13 비디오 콘텐츠 분석을 통한 데이터 수익화 컴퓨터  영상, 머신러닝 , 딥러닝을  활용해  비디오  콘텐츠를  정밀 분석한다 
파일 : how-to-use-AI-in-6industries.pdf
페이지 : 12


In [34]:
bot_message = query_result['result']
for i, doc in enumerate(query_result['source_documents']):
    bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '
    print(bot_message)

**소비자 부문 AI 활용사례**는 소비자 산업에서 AI 기술을 활용하여 운영 효율성을 높이고 가치를 창출하는 사례를 말합니다. 예를 들어, 소셜 미디어에 등장하는 비디오 데이터를 분석하여 소비자들의 관심사 및 감정을 이해하고, 이를 토대로 맞춤형 서비스를 제공하는 것이 있습니다. 또한, AI를 통해 오디오 및 비디오 데이터를 채굴하고 분석하여 고객 센터 운영이나 고객 경험 개선에 활용하는 것도 소비자 부문 AI 활용사례 중 하나입니다.

**SOURCES**
인공지능(AI) 활용서: 6대 산업별 AI 활용사례14[1] how-to-use-AI-in-6industries.pdf(13) 
**소비자 부문 AI 활용사례**는 소비자 산업에서 AI 기술을 활용하여 운영 효율성을 높이고 가치를 창출하는 사례를 말합니다. 예를 들어, 소셜 미디어에 등장하는 비디오 데이터를 분석하여 소비자들의 관심사 및 감정을 이해하고, 이를 토대로 맞춤형 서비스를 제공하는 것이 있습니다. 또한, AI를 통해 오디오 및 비디오 데이터를 채굴하고 분석하여 고객 센터 운영이나 고객 경험 개선에 활용하는 것도 소비자 부문 AI 활용사례 중 하나입니다.

**SOURCES**
인공지능(AI) 활용서: 6대 산업별 AI 활용사례14[1] how-to-use-AI-in-6industries.pdf(13) [2] how-to-use-AI-in-6industries.pdf(12) 


In [17]:
%pip install -q gradio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
import gradio as gr

# 1. PDF를 한 번만 로드하여 벡터 저장소 생성
def initialize_retriever():
    # PDF 파일을 로드하여 문서 객체로 변환
    loader = PyPDFLoader("how-to-use-AI-in-6industries.pdf")
    documents = loader.load()

    # 문서를 일정한 크기로 분할 (chunk_size=1000, 중첩 없음)
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    # 문서의 텍스트를 벡터 임베딩으로 변환
    embeddings = OpenAIEmbeddings()
    # 변환된 벡터를 Chroma 벡터 저장소에 저장
    vector_store = Chroma.from_documents(texts, embeddings)
    # 저장된 벡터를 검색할 수 있는 retriever 생성 (유사한 문서 2개 검색)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})
    return retriever

# 2. 전역 retriever 생성 (앱 시작 시 한 번만 실행)
retriever = initialize_retriever()

# 3. 채팅 응답 함수 (retriever를 재사용)
def chat_respond(message, chat_history):
    #  시스템 메시지 템플릿: 문서 요약을 기반으로 질문에 답변하도록 설정
    system_template = """Use the following pieces of context to answer the user's question shortly.
    Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), 
    use "SOURCES" in capital letters regardless of the number of sources.
    If you don't know the answer, just say that "I don't know", don't try to make up an answer.
    ----------------
    {summaries}
    
    You MUST answer in Korean and in Markdown format:
    """
    # 사용자 질문을 받아 최종 Prompt 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("human", "{question}")
    ])

    chain_type_kwargs = {
        "prompt": prompt, #  LLM이 사용할 프롬프트 지정
        "document_variable_name": "summaries", #  문서 요약 데이터를 LLM에 전달할 변수 이름
    }
    
    # OpenAI 모델 (GPT-3.5-turbo) 설정
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # RetrievalQA 체인 생성
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  #  문서를 하나의 큰 텍스트로 처리하는 방식
        retriever=retriever, #  유사 문서를 검색하는 retriever 연결
        return_source_documents=True, #  답변에 참조한 문서 정보 포함
        chain_type_kwargs=chain_type_kwargs,
        input_key="question" #  사용자 질문을 "question" 키로 전달
    )
    
    # LLM을 호출하여 질문에 대한 응답 생성
    query_result = chain.invoke({"question": message})

    # 모델의 답변을 가져오기
    bot_message = query_result['result']

    # 참조한 문서 정보를 응답에 추가
    for i, doc in enumerate(query_result['source_documents']):
        bot_message += f' [{i+1}] {doc.metadata.get("source", "Unknown")} (Page {doc.metadata.get("page", "N/A")})'

    # Gradio 채팅 기록 형식에 맞춰 응답을 저장
    chat_history.append({"role": "user", "content": message})  # 사용자 메시지 추가
    chat_history.append({"role": "assistant", "content": bot_message})  # 봇 응답 추가
    
    # 입력 창 초기화 및 갱신된 채팅 기록 반환
    return "", chat_history

# 4. # Gradio UI 생성 및 실행
with gr.Blocks() as demo:  
    # 채팅 창 (채팅 메시지를 표시하는 Gradio 컴포넌트)
    chatbot = gr.Chatbot(label="채팅창", type="messages")
    # 사용자 입력 텍스트 박스 (메시지를 입력하는 필드)
    msg = gr.Textbox(label="입력")
    # 초기화 버튼 (채팅 기록을 초기화하는 버튼)
    clear = gr.Button("초기화")

    # 사용자가 입력 필드에 메시지를 입력하면 chat_respond()가 실행됨
    #  - 입력한 메시지는 msg에서 가져옴
    #  - 기존 채팅 기록(chatbot)과 함께 전달됨
    #  - 응답을 받은 후, msg를 초기화하고 chatbot에 대화 내용을 추가
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # 초기화 버튼 클릭 시, 채팅 기록을 비우도록 설정
    clear.click(lambda: [], None, chatbot, queue=False)

# Gradio 앱 실행 (debug 모드 활성화)
demo.launch(debug=True)  

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
